In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train = pd.read_csv('/kaggle/input/llm-classification-finetuning/train.csv')
test = pd.read_csv('/kaggle/input/llm-classification-finetuning/test.csv')


In [ ]:
train

In [ ]:
test

In [ ]:
X_train = train.drop(columns=['id', 'model_a', 'model_b', 'winner_model_a', 'winner_model_b', 'winner_tie'])
y_train = train.drop(columns=['id', 'model_a', 'model_b', 'prompt', 'response_a', 'response_b'])
X_test = test.drop(columns=['id'])
ids = test['id']

In [ ]:
import torch

In [ ]:
y_train = torch.argmax(torch.tensor(np.array(y_train)), dim=1)

In [ ]:
y_train = pd.Series(y_train)
y_train

In [ ]:
X_train['overall_1'] = X_train['prompt'] + " " + X_train['response_a']
X_train['overall_2'] = X_train['prompt'] + " " + X_train['response_b']

In [ ]:
X_train.isna().sum()

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
vectorizer = TfidfVectorizer(ngram_range=(1, 2), max_features=1000)

In [ ]:
X1 = vectorizer.fit_transform(X_train['overall_1'])
X2 = vectorizer.transform(X_train['overall_2'])
X = X1 - X2
X

In [ ]:
from sklearn.model_selection import train_test_split



In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
from xgboost import XGBClassifier

In [ ]:
model = XGBClassifier()

In [ ]:
model.fit(X, y_train)


In [ ]:
X_test['overall_1'] = X_test['prompt'] + " " + X_test['response_a']
X_test['overall_2'] = X_test['prompt'] + " " + X_test['response_b']
X1_test = vectorizer.transform(X_test['overall_1'])
X2_test = vectorizer.transform(X_test['overall_2'])
X_test_aft = X1_test - X2_test

In [ ]:
pred = model.predict_proba(X_test_aft)

In [ ]:
pred

In [ ]:
pred[:, 0]

In [ ]:
ss = pd.read_csv('/kaggle/input/llm-classification-finetuning/sample_submission.csv')

ss

In [ ]:
pd.DataFrame({'id': ids, 'winner_model_a': pred[:, 0], 'winner_model_b': pred[:, 1], 'winner_tie': pred[:, 2]}).to_csv('submission.csv', index=False)